### Creating and Persisting an ML Model

In [362]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [363]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [364]:
print(df)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel freetime  goout  Dalc  Walc health 

Create a subset of features as an example.

In [365]:
include = ['studytime', 'failures','higher', 'absences', 'reason', 'health', 'age', 'activities', 'G1', 'G2', 'G3']
df.drop(columns=df.columns.difference(include), inplace=True)

def map1 (s):
    if s == 'home':
        return 0
    elif s == 'reputation':
        return 1
    elif s == 'course':
        return 2
    else:
        return 3

def map2 (s):
    if s == 'yes':
        return 0
    else:
        return 1

t1 = df.reason.tolist()
t2 = df.higher.tolist()
t3 = df.activities.tolist()
for i in range(len(t1)):
    t1[i] = map1(t1[i])
    t2[i] = map2(t2[i])
    t3[i] = map2(t3[i])
df.reason = t1
df.higher = t2
df.activities = t3

In [366]:
df.info

<bound method DataFrame.info of      age  reason  studytime  failures  activities  higher  health  absences  \
0     18       2          2         0           1       0       3         6   
1     17       2          2         0           1       0       3         4   
2     15       3          2         3           1       0       3        10   
3     15       0          3         0           0       0       5         2   
4     16       0          2         0           1       0       5         4   
..   ...     ...        ...       ...         ...     ...     ...       ...   
390   20       2          2         2           1       0       4        11   
391   17       2          1         0           1       0       2         3   
392   21       2          1         3           1       0       3         3   
393   18       2          1         0           1       0       5         0   
394   19       2          1         0           1       0       5         5   

     G1  G2  G3  
0

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [367]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [368]:
df.describe()

,age,reason,studytime,failures,activities,higher,health,absences,G1,G2,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,1.273418,2.035443,0.334177,0.491139,0.050633,3.554430,5.708861,10.908861,10.713924,10.415190,0.184810
std,1.276043,0.967060,0.839240,0.743651,0.500555,0.219525,1.390303,8.003096,3.319195,3.761505,4.581443,0.388636
min,15.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,16.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.000000,8.000000,9.000000,8.000000,0.000000
50%,17.000000,1.000000,2.000000,0.000000,0.000000,0.000000,4.000000,4.000000,11.000000,11.000000,11.000000,0.000000
75%,18.000000,2.000000,2.000000,0.000000,1.000000,0.000000,5.000000,8.000000,13.000000,13.000000,14.000000,0.000000
max,22.000000,3.000000,4.000000,3.000000,1.000000,1.000000,5.000000,75.000000,19.000000,19.000000,20.000000,1.000000


Drop the G3 score

In [369]:
include = ['studytime', 'failures','higher', 'absences', 'reason', 'health', 'age', 'G1', 'G2', 'qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

In [370]:
df.describe()

,age,reason,studytime,failures,higher,health,absences,G1,G2,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,1.273418,2.035443,0.334177,0.050633,3.554430,5.708861,10.908861,10.713924,0.184810
std,1.276043,0.967060,0.839240,0.743651,0.219525,1.390303,8.003096,3.319195,3.761505,0.388636
min,15.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,0.000000,1.000000,0.000000,0.000000,3.000000,0.000000,8.000000,9.000000,0.000000
50%,17.000000,1.000000,2.000000,0.000000,0.000000,4.000000,4.000000,11.000000,11.000000,0.000000
75%,18.000000,2.000000,2.000000,0.000000,0.000000,5.000000,8.000000,13.000000,13.000000,0.000000
max,22.000000,3.000000,4.000000,3.000000,1.000000,5.000000,75.000000,19.000000,19.000000,1.000000


Import scikit-learn and build a random forest classifer

In [371]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split

import sklearn
dependent_variable = 'qual_student'
x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
clf = rf(n_estimators = 10000)
clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=10000)

In [372]:
from sklearn.model_selection import cross_val_score

pred = clf.predict(x_test)
f1_score = sklearn.metrics.f1_score(y_test, pred, average='binary')
scores = cross_val_score(clf, x, y, cv = 5)


In [373]:
print(f1_score)
print(scores)

0.9375
[0.94936709 0.98734177 1.         0.98734177 0.97468354]


It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [374]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, '/home/yuzhouwang/17313/fall-2021-hw4-crabs-adjust-humidity/dockerfile/apps/model.pkl')

['/home/yuzhouwang/17313/fall-2021-hw4-crabs-adjust-humidity/dockerfile/apps/model.pkl']

In [375]:
query_df = pd.DataFrame({ 'age' : pd.Series(1) ,'health' : pd.Series(15) ,'absences' : pd.Series(10)})

In [376]:
pred = clf.predict(query_df)

/home/yuzhouwang/miniconda3/lib/python3.9/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- G1
- G2
- failures
- higher
- reason
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 3 features, but RandomForestClassifier is expecting 9 features as input.

In [ ]:
x

,absences,age,health
0,6,18,3
1,4,17,3
2,10,15,3
3,2,15,5
4,4,16,5
...,...,...,...
390,11,20,4
391,3,17,2
392,3,21,3
393,0,18,5


In [ ]:
type(x)

pandas.core.frame.DataFrame